In [2]:
import numpy as np
bases={'A':np.array([0,0,0,1]), 'C':np.array([0,0,1,0]), 'G':np.array([0,1,0,0]), 'T':np.array([1,0,0,0]), 'N':np.array([0,0,0,0]), 'Y':np.array([0,0,0,0]), 'R':np.array([0,0,0,0]), 'W':np.array([0,0,0,0]), 'S':np.array([0,0,0,0]), 'K':np.array([0,0,0,0])}

def one_hot_encode(seq, unique_elems,maxlen):
    seq_encoded=[]
    i=0
    for el in seq:
        # print(el)
        if i<maxlen:
            seq_encoded.append(unique_elems[el])
            i+=1
        
    return np.array(seq_encoded)

def one_hot_encode_2(y,num_classes):
    y_encoded=[]
    for el in y:
        y_encoded.append(np.eye(num_classes)[el])
    return np.array(y_encoded)



In [3]:
from Bio import SeqIO
unique_elems=dict()
x_data=[]
y_data=[]
minlen=100000
for sequence in SeqIO.parse('dataset.fasta', "fasta"):
    unique_elems[sequence.description.split()[1]+sequence.description.split()[2]]=0
    if len(sequence.seq)<minlen:
        minlen=len(sequence.seq)

# print(minlen)
# print(unique_elems.keys())
i=0
for el in unique_elems.keys():
    unique_elems[el]=i
    i+=1
# print(unique_elems)
for sequence in SeqIO.parse('dataset.fasta', "fasta"):
    x_data.append(one_hot_encode(sequence.seq,bases,minlen))
    y_data.append(unique_elems[sequence.description.split()[1]+sequence.description.split()[2]])

x_data=np.array(x_data)
y_data=one_hot_encode_2(y_data,len(unique_elems))


print(x_data.shape)
    

(118, 1110, 4)


In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D,Reshape
from tensorflow.keras.callbacks import EarlyStopping
from transformers import DistilBertTokenizer, DistilBertModel


In [17]:
model=Sequential()
model.add(Conv1D(filters=12,kernel_size=12,padding='valid',input_shape=(minlen,4), activation='relu'))
model.add(Conv1D(filters=8,kernel_size=8,padding='valid', activation='relu'))
model.add(Flatten())
model.add(Dense(120, activation='relu'))
model.add(Reshape((-1,2)))
model.add(Conv1D(filters=12,kernel_size=12,padding='valid', activation='relu'))
model.add(Conv1D(filters=8,kernel_size=8,padding='valid', activation='relu'))
model.add(Dense(120, activation='relu'))
model.add(Reshape((-1,2)))
model.add(Conv1D(filters=12,kernel_size=12,padding='valid', activation='relu'))
model.add(Flatten())
model.add(Dense(38, activation='relu'))
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='loss', patience=2, mode='min')
history = model.fit(x_data, y_data, epochs=1000, verbose=1, validation_data= None)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_42 (Conv1D)           (None, 1099, 12)          588       
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 1092, 8)           776       
_________________________________________________________________
flatten_12 (Flatten)         (None, 8736)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 120)               1048440   
_________________________________________________________________
reshape_10 (Reshape)         (None, 60, 2)             0         
_________________________________________________________________
conv1d_44 (Conv1D)           (None, 49, 12)            300       
_________________________________________________________________
conv1d_45 (Conv1D)           (None, 42, 8)           

2021-12-31 11:28:18.843426: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 1s 34ms/step - loss: 9.1644 - accuracy: 0.0593
Epoch 2/1000
4/4 [==============================] - 0s 17ms/step - loss: 8.3554 - accuracy: 0.1356
Epoch 3/1000
4/4 [==============================] - 0s 17ms/step - loss: 8.3443 - accuracy: 0.1356
Epoch 4/1000
4/4 [==============================] - 0s 16ms/step - loss: 8.3338 - accuracy: 0.1356
Epoch 5/1000
4/4 [==============================] - 0s 16ms/step - loss: 8.3231 - accuracy: 0.1356
Epoch 6/1000
4/4 [==============================] - 0s 16ms/step - loss: 8.3113 - accuracy: 0.1356
Epoch 7/1000
4/4 [==============================] - 0s 16ms/step - loss: 8.3033 - accuracy: 0.1356
Epoch 8/1000
4/4 [==============================] - 0s 16ms/step - loss: 8.2939 - accuracy: 0.1356
Epoch 9/1000
4/4 [==============================] - 0s 16ms/step - loss: 8.2839 - accuracy: 0.1356
Epoch 10/1000
4/4 [==============================] - 0s 17ms/step - loss: 8.2704 - accuracy: 0.1356
Epoch 11/1000
4/4 [===

KeyboardInterrupt: 